In [2]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
df = pd.read_csv("DataSet/ArticlesApril2018.csv")
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [5]:
print("Count of Column: ",len(df.columns))
print(df.columns)

Count of Column:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [6]:
print(df['headline'].isnull().values.any())

False


In [7]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
print("Count of Semple : ", len(headline))

Count of Semple :  1324


In [9]:
headline = [word for word in headline if word != "Unknown"]

In [10]:
print("Count of Semple : ", len(headline))

Count of Semple :  1214


In [11]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [12]:
def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print("Size of Vocabulary : ", vocab_size)

Size of Vocabulary :  3494


In [19]:
sequences= list()

for sentence in preprocessed_headline:
    
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [23]:
index_to_word = {}
for key, value in tokenizer.word_index.items():
    index_to_word[value] = key
    
print("Rank of frequency : ", index_to_word[5])

Rank of frequency :  in


In [24]:
max_len = max(len(l) for l in sequences)
print(max_len)

24


In [25]:
sequences = pad_sequences(sequences, maxlen = max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [26]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [28]:
print(X[:5])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   99  269  371 1115]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   99  269  371 1115  582]]


In [29]:
print(y[:5])

[ 269  371 1115  582   52]


In [30]:
y = to_categorical(y, num_classes=vocab_size) #One-Hot Encording

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU

In [33]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 8s - loss: 7.6973 - accuracy: 0.0296 - 8s/epoch - 33ms/step
Epoch 2/200
244/244 - 2s - loss: 7.1107 - accuracy: 0.0340 - 2s/epoch - 7ms/step
Epoch 3/200
244/244 - 2s - loss: 6.8897 - accuracy: 0.0455 - 2s/epoch - 7ms/step
Epoch 4/200
244/244 - 2s - loss: 6.6629 - accuracy: 0.0475 - 2s/epoch - 7ms/step
Epoch 5/200
244/244 - 2s - loss: 6.4107 - accuracy: 0.0511 - 2s/epoch - 8ms/step
Epoch 6/200
244/244 - 2s - loss: 6.1475 - accuracy: 0.0582 - 2s/epoch - 8ms/step
Epoch 7/200
244/244 - 2s - loss: 5.8847 - accuracy: 0.0650 - 2s/epoch - 8ms/step
Epoch 8/200
244/244 - 2s - loss: 5.6202 - accuracy: 0.0683 - 2s/epoch - 7ms/step
Epoch 9/200
244/244 - 2s - loss: 5.3634 - accuracy: 0.0773 - 2s/epoch - 8ms/step
Epoch 10/200
244/244 - 2s - loss: 5.1176 - accuracy: 0.0851 - 2s/epoch - 8ms/step
Epoch 11/200
244/244 - 2s - loss: 4.8843 - accuracy: 0.0994 - 2s/epoch - 7ms/step
Epoch 12/200
244/244 - 2s - loss: 4.6622 - accuracy: 0.1203 - 2s/epoch - 7ms/step
Epoch 13/200
244/244 - 2

Epoch 101/200
244/244 - 2s - loss: 0.3253 - accuracy: 0.9145 - 2s/epoch - 7ms/step
Epoch 102/200
244/244 - 2s - loss: 0.3170 - accuracy: 0.9130 - 2s/epoch - 7ms/step
Epoch 103/200
244/244 - 2s - loss: 0.3187 - accuracy: 0.9144 - 2s/epoch - 7ms/step
Epoch 104/200
244/244 - 2s - loss: 0.3125 - accuracy: 0.9141 - 2s/epoch - 7ms/step
Epoch 105/200
244/244 - 2s - loss: 0.3117 - accuracy: 0.9143 - 2s/epoch - 7ms/step
Epoch 106/200
244/244 - 2s - loss: 0.3141 - accuracy: 0.9132 - 2s/epoch - 7ms/step
Epoch 107/200
244/244 - 2s - loss: 0.3121 - accuracy: 0.9140 - 2s/epoch - 7ms/step
Epoch 108/200
244/244 - 2s - loss: 0.3115 - accuracy: 0.9144 - 2s/epoch - 7ms/step
Epoch 109/200
244/244 - 2s - loss: 0.3102 - accuracy: 0.9158 - 2s/epoch - 7ms/step
Epoch 110/200
244/244 - 2s - loss: 0.3077 - accuracy: 0.9132 - 2s/epoch - 7ms/step
Epoch 111/200
244/244 - 2s - loss: 0.3027 - accuracy: 0.9141 - 2s/epoch - 7ms/step
Epoch 112/200
244/244 - 2s - loss: 0.3069 - accuracy: 0.9146 - 2s/epoch - 7ms/step
Epoc

Epoch 200/200
244/244 - 2s - loss: 0.2979 - accuracy: 0.9116 - 2s/epoch - 8ms/step


In [46]:
current_word = 'what'

encoded = tokenizer.texts_to_sequences([current_word])[0]
encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
result = model.predict(encoded, verbose=0)
result = np.argmax(result, axis=1)

for word, index in tokenizer.word_index.items():
    if index == result:
        break

print(word)

the


In [49]:
current_word = 'the'

encoded = tokenizer.texts_to_sequences([current_word])[0]
encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
result = model.predict(encoded, verbose=0)
result = np.argmax(result, axis=1)

for word, index in tokenizer.word_index.items():
    if index == result:
        break

print(word)

americans


In [50]:
current_word = 'what the'

encoded = tokenizer.texts_to_sequences([current_word])[0]
encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
result = model.predict(encoded, verbose=0)
result = np.argmax(result, axis=1)

for word, index in tokenizer.word_index.items():
    if index == result:
        break

print(word)

heck


In [51]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''
    
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
        
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            if index == result:
                break
        
        current_word = current_word + ' '  + word
        sentence = sentence + ' ' + word
        
    sentence = init_word + sentence
    
    return sentence

In [57]:
print(sentence_generation(model, tokenizer, 'what', 20))

what the heck is that abou excited they are was guns still alive he say and new york gallery side the


In [59]:
print(sentence_generation(model, tokenizer, 'the', 25))

the americans season 6 episode 4 recap the birth of a honey pot in prize into abuse coverage stars in her university war been a missed
